-Download Shapefile of the Sector Boundary of the country selected.                      
-This acts as a boundry input to the Google Static maps API.

In [ ]:
def get_shp_extent(shp_file):
    print("shp_file_ext")
    inDriver = ogr.GetDriverByName("ESRI Shapefile")
    inDataSource = inDriver.Open(inShapefile, 0)
    inLayer = inDataSource.GetLayer()
    extent = inLayer.GetExtent()
    # x_min_shp, x_max_shp, y_min_shp, y_max_shp = extent
    return extent

In [ ]:
from retrying import retry
import urllib.request
import matplotlib.pyplot as plt

@retry(wait_exponential_multiplier=1000, wait_exponential_max=3600000)
def save_img(url, file_path, file_name):
    image = plt.imread(url)
    if np.array_equal(image[:,:10,:],image[:,10:20,:]):
        print("=======")
        pass
    else:
        print('---')
        plt.imsave(file_path + file_name, image[50:450, :, :])

inShapefile = "data/shp/Sector_Boundary_2012/Sector_Boundary_2012.shp"
x_min_shp, x_max_shp, y_min_shp, y_max_shp = get_shp_extent(inShapefile)

left_idx, top_idx = get_cell_idx(x_min_shp, y_max_shp, top_left_x_coords, top_left_y_coords)
right_idx, bottom_idx = get_cell_idx(x_max_shp, y_min_shp, top_left_x_coords, top_left_y_coords)

key = ''#enter the API key here 
m = 1
for i in range(left_idx, right_idx + 1):
    for j in range(top_idx, bottom_idx + 1):
        lon = centroid_x_coords[i]
        lat = centroid_y_coords[j]
        print(lon,lat)
        url = 'https://maps.googleapis.com/maps/api/staticmap?center=' + str(lat) + ',' + \
               str(lon) + '&zoom=16&size=400x400&maptype=satellite&key=' + key
        lightness = bands_data[j, i, 0]
        file_path = 'google_image/' + str(lightness) + '/'
        if not os.path.isdir(file_path):
            os.makedirs(file_path)
        file_name = str(i) + '_' + str(j) +'.jpg'
        #save_img(url, file_path, file_name)
        print(file_path, file_name)
        if m % 100 == 0:
            print(m)
        m += 1
